# Step1: 추천 후보군 생성
1. 라우터로 유저 고유 아이디 들어옴
2. 이 고유 아이디 들어오면 장우님 뉴스 로그 api로 
{
      "timestamp": "2025-06-18T17:22:03",
      "timestampRaw": "2025-06-18 17:22:03",
      "thread": "http-nio-8080-exec-3",
      "level": "INFO",
      "logger": "c.n.n.a.n.v2.impl.ml.GetDetailNewsV2",
      "message": "2025-06-18 17:22:03 [http-nio-8080-exec-3] [INFO ] c.n.n.a.n.v2.impl.ml.GetDetailNewsV2 - [memberId : anonymous]  [newsId : 20250618_28735495]",
      "memberId": "anonymous",
      "newsId": "20250618_28735495" -> 이게 클릭 뉴스 아이디 됨
    },
이거 받아옴
뉴스 상세 메타데이터 조회로 데이터 미리 불러와서 클릭 뉴스 df 만들어 놓기 
{
  "news_id": "string",
  "summary": "string",
  "stock_list": [],
  "stock_list_view": [],
  "industry_list": [],
  "impact_score": 0
}

2-1. 뉴스 로그 api 없는 경우도 고려
4. 주요 뉴스 목록 조회로 3개월치 받아오기 -> 얘가 후보 뉴스 아이디
5. onnx 모델 불러와서 돌려 two-tower -> 클릭 뉴스 임베딩(n), 후보 뉴스 임베딩(n) 리턴은 후보뉴스 여러개의 재정렬된 리스트
6. 10개 걸러서 news_id 빼기

# Step2: 추천 후보군 리랭킹
1. 유저 아이디 이용해서 유저 정보 조회 api로 userPnl, asset, investScore 가져오기
2. 후보군 뉴스 상세 메타데이터 조회로 스톡리스트, 인더스트리 리스트, 임팩트 스코어, 토픽 db에서 끌고와서 붙이고 가장 큰값으로 메인토픽도 붙이기
4. 나온 결과로 내 모델에 넣어서 리랭킹 4개


# Step1

----------------------------------------------

1. 유저 고유아이디 수집

라우터 만들어서 받아주세요이

2. 뉴스 로그 api

In [133]:
import requests
import pandas as pd
import time
import onnxruntime as ort
from collections import Counter
import numpy as np
import nest_asyncio  # Jupyter 환경에서만 필요
import asyncio
import httpx
import joblib
import os
import sys

In [103]:
import requests
import pandas as pd

def collect_member_news_data(member_id: str, start_date: str, end_date: str) -> (list, pd.DataFrame):
    """
    특정 멤버의 뉴스 로그 데이터 수집
    :param member_id: 멤버 ID (예: "anonymous", "user123")
    :param start_date: 조회 시작 날짜 (YYYY-MM-DD)
    :param end_date: 조회 종료 날짜 (YYYY-MM-DD)
    :return: 
        - unique_news_ids: 중복 제거된 newsId 목록
        - click_log_df: 원본 로그 데이터 DataFrame (news_id 컬럼 포함)
    """
    API_BASE_URL = "http://43.200.17.139:8080"
    NEWS_LOGS_ENDPOINT = "/api/newsLogs"
    url = API_BASE_URL + NEWS_LOGS_ENDPOINT
    params = {
        "startDate": start_date,
        "endDate": end_date,
        "memberId": member_id
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        api_response = response.json()
    except Exception as e:
        print(f"API 호출 실패: {str(e)}")
        return [], pd.DataFrame()
    
    if not api_response.get("Success", False):
        return [], pd.DataFrame()
    
    # 원본 데이터 추출
    log_data = api_response.get("data", [])
    
    # 1. 뉴스 ID 추출 및 중복 제거
    news_ids = [log.get("newsId") for log in log_data if "newsId" in log]
    unique_news_ids = list(set(news_ids))
    
    # 2. 원본 로그 데이터를 DataFrame으로 변환
    click_log_df = pd.DataFrame(log_data)
    
    # 3. 컬럼명 변경: newsId -> news_id
    if 'newsId' in click_log_df.columns:
        click_log_df = click_log_df.rename(columns={'newsId': 'news_id'})
    
    print(f"멤버 '{member_id}': {len(log_data)}개 로그, {len(unique_news_ids)}개 고유 뉴스")
    return unique_news_ids, click_log_df


In [ ]:
# # 파라미터 설정
# member_id = "anonymous"  # 또는 실제 멤버 ID
# start_date = "2025-06-01"
# end_date = "2025-06-19"
    
# # 뉴스 ID 수집
# unique_news_ids, click_log_df = collect_member_news_data(member_id, start_date, end_date)

# # 결과 활용
# print("로그 데이터 샘플:")
# click_log_df

멤버 'anonymous': 169개 로그, 46개 고유 뉴스
로그 데이터 샘플:


,timestamp,timestampRaw,thread,level,logger,message,memberId,news_id
0,2025-06-18T17:21:52,2025-06-18 17:21:52,http-nio-8080-exec-3,INFO,c.n.n.a.n.v2.impl.ml.GetDetailNewsV2,2025-06-18 17:21:52 [http-nio-8080-exec-3] [IN...,anonymous,20250513_0092
1,2025-06-18T17:22:03,2025-06-18 17:22:03,http-nio-8080-exec-3,INFO,c.n.n.a.n.v2.impl.ml.GetDetailNewsV2,2025-06-18 17:22:03 [http-nio-8080-exec-3] [IN...,anonymous,20250618_28735495
2,2025-06-18T17:26:12,2025-06-18 17:26:12,http-nio-8080-exec-4,INFO,c.n.n.a.n.v2.impl.ml.GetDetailNewsV2,2025-06-18 17:26:12 [http-nio-8080-exec-4] [IN...,anonymous,20241015_0007
3,2025-06-18T17:26:54,2025-06-18 17:26:54,http-nio-8080-exec-6,INFO,c.n.n.a.n.v2.impl.ml.GetDetailNewsV2,2025-06-18 17:26:54 [http-nio-8080-exec-6] [IN...,anonymous,20240529_0117
4,2025-06-18T17:29:20,2025-06-18 17:29:20,http-nio-8080-exec-3,INFO,c.n.n.a.n.v2.impl.ml.GetDetailNewsV2,2025-06-18 17:29:20 [http-nio-8080-exec-3] [IN...,anonymous,20240725_0183
...,...,...,...,...,...,...,...,...
164,2025-06-19T22:35:39,2025-06-19 22:35:39,http-nio-8080-exec-3,INFO,c.n.n.a.n.v2.impl.ml.GetDetailNewsV2,2025-06-19 22:35:39 [http-nio-8080-exec-3] [IN...,anonymous,20250523_0004
165,2025-06-19T22:50:57,2025-06-19 22:50:57,http-nio-8080-exec-1,INFO,c.n.n.a.n.v2.impl.ml.GetDetailNewsV2,2025-06-19 22:50:57 [http-nio-8080-exec-1] [IN...,anonymous,20250619_43379034
166,2025-06-19T22:51:03,2025-06-19 22:51:03,http-nio-8080-exec-2,INFO,c.n.n.a.n.v2.impl.ml.GetDetailNewsV2,2025-06-19 22:51:03 [http-nio-8080-exec-2] [IN...,anonymous,20250523_0004
167,2025-06-19T23:22:41,2025-06-19 23:22:41,http-nio-8080-exec-9,INFO,c.n.n.a.n.v2.impl.ml.GetDetailNewsV2,2025-06-19 23:22:41 [http-nio-8080-exec-9] [IN...,anonymous,20250619_11889916


이 뉴스 아이디 리스트 이용해서 본문 뽑아야됨

In [105]:
import requests
import pandas as pd

def get_news_metadata_df(news_ids: list) -> pd.DataFrame:
    """
    뉴스 ID 목록을 받아 각 뉴스의 메타데이터를 조회하여 DataFrame으로 반환
    :param news_ids: 뉴스 ID 문자열 리스트 (예: ['20250513_0092', '20250618_28735495'])
    :return: 뉴스 메타데이터 DataFrame
    """
    API_BASE_URL = "http://3.37.207.16:8000"
    METADATA_ENDPOINT = "/news/v2/{news_id}/metadata"
    
    records = []
    
    # news_ids가 유효한 리스트인지 확인
    if not isinstance(news_ids, list) or not all(isinstance(i, str) for i in news_ids):
        print("잘못된 news_ids 형식: 반드시 문자열 리스트여야 합니다")
        return pd.DataFrame()
    
    for news_id in news_ids:
        # URL 정상 생성 확인
        url = API_BASE_URL + METADATA_ENDPOINT.format(news_id=news_id)
        try:
            response = requests.get(url)
            response.raise_for_status()  # 4xx/5xx 오류 시 예외 발생
            meta = response.json()
            records.append(meta)
            print(f"{news_id} 메타데이터 조회 성공")
        except requests.exceptions.HTTPError as e:
            print(f"{news_id} 처리 중 HTTP 오류 ({e.response.status_code}): {e}")
        except Exception as e:
            print(f"{news_id} 처리 중 오류: {e}")
    
    return pd.DataFrame(records)



In [ ]:
# # DataFrame 생성 예시
# clicked_df = get_news_metadata_df(unique_news_ids)

# clicked_df.info()

20240715_0269 메타데이터 조회 성공
20250618_24658192 처리 중 HTTP 오류 (404): 404 Client Error: Not Found for url: http://3.37.207.16:8000/news/v2/20250618_24658192/metadata
20250619_08305615 처리 중 HTTP 오류 (404): 404 Client Error: Not Found for url: http://3.37.207.16:8000/news/v2/20250619_08305615/metadata
20250523_0015 메타데이터 조회 성공
20241021_0109 메타데이터 조회 성공
20250523_0004 메타데이터 조회 성공
20250619_02264772 처리 중 HTTP 오류 (404): 404 Client Error: Not Found for url: http://3.37.207.16:8000/news/v2/20250619_02264772/metadata
20250523_0012 메타데이터 조회 성공
20250523_0067 메타데이터 조회 성공
20250618_28735495 처리 중 HTTP 오류 (404): 404 Client Error: Not Found for url: http://3.37.207.16:8000/news/v2/20250618_28735495/metadata
20250523_0011 메타데이터 조회 성공
20250523_0050 메타데이터 조회 성공
20250619_52456644 처리 중 HTTP 오류 (404): 404 Client Error: Not Found for url: http://3.37.207.16:8000/news/v2/20250619_52456644/metadata
20250618_74048781 처리 중 HTTP 오류 (404): 404 Client Error: Not Found for url: http://3.37.207.16:8000/news/v2/20250618_740487

4. 주요 뉴스 목록 조회로 3개월치 받아오기 -> 얘가 후보 뉴스 아이디 (candi_df)

In [107]:
def get_highlight_data_df(start_datetime: str, end_datetime: str, limit: int = 50, stock_list: str = "") -> pd.DataFrame:
    """
    하이라이트 데이터 API에서 데이터 조회 후 DataFrame으로 반환
    :param start_datetime: 시작일시 (예: "2025-02-15T00:00:00")
    :param end_datetime: 종료일시 (예: "2025-05-16T00:00:00")
    :param limit: 조회 개수 제한 (기본값 50)
    :param stock_list: 종목코드 문자열 (예: "005930,000660")
    :return: 하이라이트 데이터 DataFrame
    """
    API_BASE_URL = "http://3.37.207.16:8000"
    HIGHLIGHTDATA_ENDPOINT = "/news/v2/highlights"
    params = {
        "start_datetime": start_datetime,
        "end_datetime": end_datetime,
        "limit": limit,
        "stock_list": stock_list
    }
    try:
        response = requests.get(API_BASE_URL + HIGHLIGHTDATA_ENDPOINT, params=params)
        response.raise_for_status()
        data = response.json()
        df = pd.DataFrame(data)
        print(f"주요 뉴스 {len(df)}건 조회 완료")
        return df
    except Exception as e:
        print(f"API 호출 또는 데이터 처리 중 오류: {e}")
        return pd.DataFrame()


In [ ]:
# candi_df = get_highlight_data_df(
#     start_datetime="2025-02-15T00:00:00",
#     end_datetime="2025-05-16T00:00:00",
#     limit=50
# )
# candi_df


주요 뉴스 50건 조회 완료


,news_id,wdate,title,image,press,summary,impact_score,url
0,20250429_0099,2025-04-29T10:05:00,"한덕수 대선 출마 임박설에 관련주 급등…일정실업, 이틀 연속 上",https://imgnews.pstatic.net/image/009/2025/04/...,매일경제,한덕수 대통령 권한대행 국무총리의 대선 출마가 임박한 것으로 알려지며 일정실업은 고...,1.44,https://n.news.naver.com/mnews/article/009/000...
1,20250414_0083,2025-04-14T09:42:00,오세훈 대선 불출마에 테마주 '와르르',https://imgnews.pstatic.net/image/215/2025/04/...,한국경제TV,윤석열 전 대통령 탄핵 이후 치러지는 조기 대선 유력 후보로 꼽혀 오세훈 서울시장이...,0.86,https://n.news.naver.com/mnews/article/215/000...
2,20250513_0089,2025-05-13T09:31:00,"한진칼, '쩜상'…추격매수냐 vs 익절이냐 '분수령'",https://imgnews.pstatic.net/image/215/2025/05/...,한국경제TV,호반건설이 지난 3월 한진칼 주주총회에 참석해 이사 보수 한도를 90억원에서 120...,0.78,https://n.news.naver.com/mnews/article/215/000...
3,20250513_0148,2025-05-13T00:25:00,"호반건설, 한진칼 지분 더 샀다…조원태와 1.5%P差",https://ssl.pstatic.net/static.news/image/news...,한국경제,호반건설이 12일 한진칼 보유 지분을 17.44%에서 18.46%로 1.02%포인트...,0.78,https://n.news.naver.com/mnews/article/015/000...
4,20250513_0094,2025-05-13T09:16:00,"호반건설 지분 매입에, 한진칼 장 초반 약 30% 강세 [특징주]",https://imgnews.pstatic.net/image/014/2025/05/...,파이낸셜뉴스,호반건설이 한진그룹 지주사 한진칸 지분율을 18% 이상으로 높였다는 소식에 한진칼이...,0.78,https://n.news.naver.com/mnews/article/014/000...
5,20250513_0092,2025-05-13T09:17:00,호반건설 한진그룹 경영 개입할까?…한진칼 상한가행,https://imgnews.pstatic.net/image/008/2025/05/...,머니투데이,호반건설은 한진칼 보유지분을 1%p(포인트)가량 추가로 사들여 17.44%에서 18...,0.78,https://n.news.naver.com/mnews/article/008/000...
6,20250513_0085,2025-05-13T09:53:00,"한진칼, 호반건설 지분 추가매입 소식에 ‘上’…경영권 분쟁 가능성",https://imgnews.pstatic.net/image/009/2025/05/...,매일경제,호반건설이 대한항공을 거느린 한진그룹 지주사인 한진칼 지분을 추가로 매입했다는 소식...,0.78,https://n.news.naver.com/mnews/article/009/000...
7,20250513_0041,2025-05-13T15:00:00,국내 유일 대형항공사 대한한공 경영권 분쟁 가능성에 ··· 대한항공 모회사 한진칼 ...,https://imgnews.pstatic.net/image/009/2025/05/...,매일경제,호반건설이 한진칼 지분을 추가로 매입해 지분율(특수관계인 포함)이 17.44%에서 ...,0.78,https://n.news.naver.com/mnews/article/009/000...
8,20250417_0108,2025-04-17T08:16:00,"제이준코스메틱, 프리미엄 'PDRN 포슬린 앰플' 신제품 출시…연어 DNA 성분 탑재",https://imgnews.pstatic.net/image/277/2025/04/...,아시아경제,K-뷰티기업 제이준코스메틱은 연어나 송어의 생식세포에서 DNA 조각을 추출한 물질인...,0.75,https://n.news.naver.com/mnews/article/277/000...
9,20250417_0067,2025-04-17T10:01:00,"제이준코스메틱, 프리미엄 'PDRN 포슬린 앰플' 신제품 출시",https://imgnews.pstatic.net/image/008/2025/04/...,머니투데이,제이준코스메틱은 연어나 송어의 생식세포에서 DNA 조각을 추출한 물질인 PDRN(폴...,0.75,https://n.news.naver.com/mnews/article/008/000...


In [ ]:
# candi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   news_id       50 non-null     object 
 1   wdate         50 non-null     object 
 2   title         50 non-null     object 
 3   image         50 non-null     object 
 4   press         50 non-null     object 
 5   summary       50 non-null     object 
 6   impact_score  50 non-null     float64
 7   url           50 non-null     object 
dtypes: float64(1), object(7)
memory usage: 3.3+ KB


5. 임베딩 -> onnx 모델 불러와서 돌려 two-tower -> 클릭 뉴스 임베딩(n), 후보 뉴스 임베딩(n) 리턴은 후보뉴스 여러개의 재정렬된 리스트

In [109]:
def add_embeddings_to_dfs(clicked_df: pd.DataFrame, candi_df: pd.DataFrame, 
                           batch_size: int = 20, sleep_time: float = 0.3) -> (pd.DataFrame, pd.DataFrame):
    """
    두 DataFrame에 요약문 임베딩을 추가하여 반환
    :param clicked_df: 클릭한 뉴스 데이터프레임 (summary 컬럼 필수)
    :param candi_df: 후보 뉴스 데이터프레임 (summary 컬럼 필수)
    :param batch_size: 배치 처리 크기 (기본값 20)
    :param sleep_time: 배치 간 간격 (초, 기본값 0.3)
    :return: 임베딩이 추가된 clicked_df, candi_df 튜플
    """
    # API 설정
    API_BASE_URL = "http://15.165.211.100:8000"
    HIGHLIGHTDATA_ENDPOINT = "/plm/embedding"
    
    # 배치 임베딩 처리 함수
    def get_embedding_batch(article_list):
        """문장 리스트를 배치로 임베딩 처리"""
        url = API_BASE_URL + HIGHLIGHTDATA_ENDPOINT
        headers = {"Content-Type": "application/json"}
        
        all_embeddings = []
        for i in range(0, len(article_list), batch_size):
            batch = article_list[i:i+batch_size]
            try:
                response = requests.post(url, json={"articles": batch})
                response.raise_for_status()
                result = response.json()
                batch_embeddings = result.get("embeddings", [])
                all_embeddings.extend(batch_embeddings)
                print(f"배치 {i}-{i+len(batch)}: {len(batch_embeddings)}개 임베딩 성공")
            except Exception as e:
                print(f"배치 {i}-{i+len(batch)} 실패: {str(e)}")
                all_embeddings.extend([None] * len(batch))
            time.sleep(sleep_time)
        return all_embeddings

    # 요약문 추출
    clicked_summary_list = clicked_df['summary'].tolist()
    candi_summary_list = candi_df['summary'].tolist()

    # 임베딩 처리
    clicked_embeddings = get_embedding_batch(clicked_summary_list)
    candi_embeddings = get_embedding_batch(candi_summary_list)

    # 임베딩 결과 추가
    clicked_df = clicked_df.copy()
    candi_df = candi_df.copy()
    clicked_df["embedding"] = clicked_embeddings
    candi_df["embedding"] = candi_embeddings

    # 결과 요약
    print(f"clicked_df: {len(clicked_df)}개 중 {len([e for e in clicked_embeddings if e is not None])}개 임베딩 성공")
    print(f"candi_df: {len(candi_df)}개 중 {len([e for e in candi_embeddings if e is not None])}개 임베딩 성공")
    
    return clicked_df, candi_df


In [110]:
# # 함수 실행
# clicked_df, candi_df = add_embeddings_to_dfs(
#     clicked_df, 
#     candi_df,
#     batch_size=30,   # 배치 크기 조정
#     sleep_time=0.2   # 요청 간격 조정
# )

# # 결과 확인
# print(clicked_df[['news_id', 'embedding']].head())
# print(candi_df[['news_id', 'embedding']].head())


배치 0-27: 27개 임베딩 성공
배치 0-30: 30개 임베딩 성공
배치 30-50: 20개 임베딩 성공
clicked_df: 27개 중 27개 임베딩 성공
candi_df: 50개 중 50개 임베딩 성공
         news_id                                          embedding
0  20240715_0269  [-1.071226954460144, 0.14030209183692932, -0.3...
1  20250523_0015  [-0.6171276569366455, -0.68632972240448, -0.65...
2  20241021_0109  [-0.19199156761169434, -0.7358613014221191, 0....
3  20250523_0004  [-0.27986904978752136, -0.5640870332717896, -0...
4  20250523_0012  [-0.7164637446403503, -0.4399840831756592, 0.1...
         news_id                                          embedding
0  20250429_0099  [-0.45991164445877075, -1.051605224609375, -0....
1  20250414_0083  [-0.1376427561044693, -0.9182015657424927, -0....
2  20250513_0089  [-0.3865353465080261, -0.9261698126792908, -0....
3  20250513_0148  [-0.5047512650489807, 0.12863653898239136, -0....
4  20250513_0094  [-1.0296272039413452, -0.6138355135917664, 0.1...


광일님 여기 진행하실때 onnx 모델 경로 꼭 수정해주세요!

In [111]:
def run_inference(
    clicked_df: pd.DataFrame, 
    candi_df: pd.DataFrame, 
    model_path: str = "C:/Users/user/fin_project/modelapi/models/two_tower_model.onnx",  # 기본값 설정
    num_clicked: int = 3, 
    num_candidates: int = 5, 
    top_k: int = 3
):
    """
    ONNX 모델을 사용한 추천 시스템 추론 및 결과 처리
    :param clicked_df: 클릭한 뉴스 DataFrame (embedding 컬럼 필수)
    :param candi_df: 후보 뉴스 DataFrame (embedding, news_id 컬럼 필수)
    :param model_path: ONNX 모델 파일 경로 (기본값 제공)
    :param num_clicked: 사용할 클릭한 뉴스 개수
    :param num_candidates: 사용할 후보 뉴스 개수
    :param top_k: 상위 추천 개수
    :return: (scores, top_indices, final_news_ids, appear_counts, appear_count_map)
    """
    # ONNX 모델 로드
    session_options = ort.SessionOptions()
    session_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
    session = ort.InferenceSession(model_path, session_options)
    
    # 임베딩 데이터 추출 및 전처리
    clicked_embeddings_np = np.array(clicked_df["embedding"].tolist(), dtype=np.float32)
    candi_embeddings_np = np.array(candi_df["embedding"].tolist(), dtype=np.float32)
    
    # 입력 데이터 형식 조정
    clicked_input = clicked_embeddings_np[:num_clicked].reshape(1, num_clicked, 768)
    candidate_input = candi_embeddings_np[:num_candidates].reshape(1, num_candidates, 768)
    
    # 모델 추론 실행
    inputs = {
        "clicked": clicked_input,
        "candidates": candidate_input
    }
    scores = session.run(None, inputs)[0]
    
    # Top-K 후보 선정
    top_indices = np.argsort(-scores, axis=2)[:, :, :top_k]
    flattened = top_indices.squeeze(0).flatten()
    
    # 유니크 후보군 및 등장 빈도 계산
    unique_candidates = np.unique(flattened)
    counts = Counter(flattened)
    
    # 실제 뉴스 ID 매핑
    candidate_indices = unique_candidates.tolist()
    final_news_ids = candi_df.iloc[candidate_indices]['news_id'].tolist()
    appear_counts = [counts[idx] for idx in candidate_indices]
    appear_count_map = dict(zip(final_news_ids, appear_counts))
    
    return scores, top_indices, final_news_ids, appear_counts, appear_count_map


In [ ]:
# # 함수 실행
# results = run_inference(
#     clicked_df=clicked_df,
#     candi_df=candi_df,
#     num_clicked=len(clicked_df),
#     num_candidates=len(candi_df),
#     top_k=3
# )

# # 결과 언패킹
# scores, top_indices, final_news_ids, appear_counts, appear_count_map = results

# # 결과 확인
# print("실제 뉴스ID 리스트:", final_news_ids)
# print("가중치 리스트:", appear_counts)
# print("뉴스ID ➝ 등장 횟수 dict:", appear_count_map)



실제 뉴스ID 리스트: ['20250513_0089', '20250513_0148', '20250513_0085', '20250513_0041', '20250430_0098', '20250430_0083', '20250408_0068', '20250331_0086', '20250326_0027', '20250307_0027', '20250422_0105', '20250219_0175', '20250509_0005', '20250424_0028', '20250219_0116', '20250219_0162', '20250409_0137']
가중치 리스트: [1, 6, 1, 4, 10, 18, 2, 1, 1, 11, 8, 2, 5, 2, 2, 1, 6]
뉴스ID ➝ 등장 횟수 dict: {'20250513_0089': 1, '20250513_0148': 6, '20250513_0085': 1, '20250513_0041': 4, '20250430_0098': 10, '20250430_0083': 18, '20250408_0068': 2, '20250331_0086': 1, '20250326_0027': 1, '20250307_0027': 11, '20250422_0105': 8, '20250219_0175': 2, '20250509_0005': 5, '20250424_0028': 2, '20250219_0116': 2, '20250219_0162': 1, '20250409_0137': 6}


# Step2

-----------------------------------------------

In [ ]:
# click_log_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169 entries, 0 to 168
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   timestamp     169 non-null    object
 1   timestampRaw  169 non-null    object
 2   thread        169 non-null    object
 3   level         169 non-null    object
 4   logger        169 non-null    object
 5   message       169 non-null    object
 6   memberId      169 non-null    object
 7   newsId        169 non-null    object
dtypes: object(8)
memory usage: 10.7+ KB


1. 유저 아이디 이용해서 유저 정보 조회 api로 userPnl, asset, investScore 가져오기

지금 아마 이 부분이 api 오류날것입니다.. 0620 오후 5시 기준

In [113]:
def fetch_userinfo_and_merge(click_log_df: pd.DataFrame) -> pd.DataFrame:
    """
    API를 사용해 사용자 정보 조회 후 click_log_df와 병합
    :param click_log_df: 클릭 로그 데이터프레임 (memberId 컬럼 필수)
    :return: 병합된 데이터프레임
    """
    # 1. 컬럼 확인 및 정리
    if 'memberId' not in click_log_df.columns:
        if 'user_id' in click_log_df.columns:
            click_log_df = click_log_df.rename(columns={'user_id': 'memberId'})
        else:
            raise ValueError("DataFrame에 'memberId' 또는 'user_id' 컬럼이 필요합니다")
    
    # 2. 고유 memberId 추출
    unique_member_ids = click_log_df['memberId'].unique()
    
    # 3. API 호출로 유저 정보 수집
    base_url = "http://43.200.17.139:8080/api/v1/userinfo/"
    user_info_list = []
    
    for member_id in unique_member_ids:
        # "anonymous"는 기본값 처리
        if member_id == "anonymous":
            user_info_list.append({
                'memberId': member_id,
                'username': 'anonymous',
                'userPnl': 0,
                'asset': 0,
                'investScore': 0,
                'memberStocks': []
            })
            continue
            
        url = base_url + str(member_id)
        try:
            response = requests.get(url)
            response.raise_for_status()
            user_data = response.json()
            user_data['memberId'] = member_id  # 병합을 위해 추가
            user_info_list.append(user_data)
            print(f"✅ 사용자 {member_id} 정보 조회 성공")
        except Exception as e:
            print(f"❌ 사용자 {member_id} 정보 조회 실패: {str(e)}")
            # 에러 발생 시 기본 데이터 생성
            user_info_list.append({
                'memberId': member_id,
                'username': 'unknown',
                'userPnl': 0,
                'asset': 0,
                'investScore': 0,
                'memberStocks': []
            })
    
    # 4. 유저 정보 DataFrame 생성
    user_df = pd.DataFrame(user_info_list)
    
    # 5. 병합 (left join)
    merged_df = pd.merge(click_log_df, user_df, on='memberId', how='left')
    
    return merged_df


In [ ]:
# merged_df = fetch_userinfo_and_merge(click_log_df)
# merged_df

,timestamp,timestampRaw,thread,level,logger,message,memberId,news_id,username,userPnl,asset,investScore,memberStocks
0,2025-06-18T17:21:52,2025-06-18 17:21:52,http-nio-8080-exec-3,INFO,c.n.n.a.n.v2.impl.ml.GetDetailNewsV2,2025-06-18 17:21:52 [http-nio-8080-exec-3] [IN...,anonymous,20250513_0092,anonymous,0,0,0,[]
1,2025-06-18T17:22:03,2025-06-18 17:22:03,http-nio-8080-exec-3,INFO,c.n.n.a.n.v2.impl.ml.GetDetailNewsV2,2025-06-18 17:22:03 [http-nio-8080-exec-3] [IN...,anonymous,20250618_28735495,anonymous,0,0,0,[]
2,2025-06-18T17:26:12,2025-06-18 17:26:12,http-nio-8080-exec-4,INFO,c.n.n.a.n.v2.impl.ml.GetDetailNewsV2,2025-06-18 17:26:12 [http-nio-8080-exec-4] [IN...,anonymous,20241015_0007,anonymous,0,0,0,[]
3,2025-06-18T17:26:54,2025-06-18 17:26:54,http-nio-8080-exec-6,INFO,c.n.n.a.n.v2.impl.ml.GetDetailNewsV2,2025-06-18 17:26:54 [http-nio-8080-exec-6] [IN...,anonymous,20240529_0117,anonymous,0,0,0,[]
4,2025-06-18T17:29:20,2025-06-18 17:29:20,http-nio-8080-exec-3,INFO,c.n.n.a.n.v2.impl.ml.GetDetailNewsV2,2025-06-18 17:29:20 [http-nio-8080-exec-3] [IN...,anonymous,20240725_0183,anonymous,0,0,0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,2025-06-19T22:35:39,2025-06-19 22:35:39,http-nio-8080-exec-3,INFO,c.n.n.a.n.v2.impl.ml.GetDetailNewsV2,2025-06-19 22:35:39 [http-nio-8080-exec-3] [IN...,anonymous,20250523_0004,anonymous,0,0,0,[]
165,2025-06-19T22:50:57,2025-06-19 22:50:57,http-nio-8080-exec-1,INFO,c.n.n.a.n.v2.impl.ml.GetDetailNewsV2,2025-06-19 22:50:57 [http-nio-8080-exec-1] [IN...,anonymous,20250619_43379034,anonymous,0,0,0,[]
166,2025-06-19T22:51:03,2025-06-19 22:51:03,http-nio-8080-exec-2,INFO,c.n.n.a.n.v2.impl.ml.GetDetailNewsV2,2025-06-19 22:51:03 [http-nio-8080-exec-2] [IN...,anonymous,20250523_0004,anonymous,0,0,0,[]
167,2025-06-19T23:22:41,2025-06-19 23:22:41,http-nio-8080-exec-9,INFO,c.n.n.a.n.v2.impl.ml.GetDetailNewsV2,2025-06-19 23:22:41 [http-nio-8080-exec-9] [IN...,anonymous,20250619_11889916,anonymous,0,0,0,[]


2. 후보군 뉴스 상세 메타데이터 조회로 스톡리스트, 인더스트리 리스트, 임팩트 스코어, 토픽 db에서 끌고와서 붙이고 가장 큰값으로 메인토픽도 붙이기

In [115]:
def get_metadata_df(news_ids: list) -> pd.DataFrame:
    """
    뉴스 ID 리스트를 받아 비동기로 메타데이터 수집 후 DataFrame 반환
    :param news_ids: 뉴스 ID 문자열 리스트
    :return: 메타데이터 DataFrame (news_id, summary, stock_list_view, error 컬럼 포함)
    """
    # API 설정
    API_BASE_URL = "http://3.37.207.16:8000"
    METADATA_ENDPOINT = "/news/v2/{news_id}/metadata"
    
    # Jupyter 환경 호환
    try:
        nest_asyncio.apply()
    except:
        pass
    
    async def fetch_single_metadata(news_id: str) -> dict:
        """개별 뉴스 메타데이터 비동기 조회"""
        url = f"{API_BASE_URL}{METADATA_ENDPOINT.format(news_id=news_id)}"
        async with httpx.AsyncClient(timeout=10.0) as client:
            try:
                response = await client.get(url)
                response.raise_for_status()
                return response.json() | {"news_id": news_id}
            except httpx.HTTPStatusError as e:
                return {
                    "news_id": news_id,
                    "error": f"HTTP {e.response.status_code}",
                    "detail": str(e)
                }
            except Exception as e:
                return {
                    "news_id": news_id,
                    "error": "ConnectionError",
                    "detail": str(e)
                }
    
    async def fetch_all_metadata():
        """모든 뉴스 메타데이터 병렬 조회"""
        tasks = [fetch_single_metadata(nid) for nid in news_ids]
        return await asyncio.gather(*tasks)
    
    # 메타데이터 수집 실행
    loop = asyncio.get_event_loop()
    metadata_list = loop.run_until_complete(fetch_all_metadata())
    
    # DataFrame 변환
    processed_data = []
    for item in metadata_list:
        if "error" in item:
            row = {
                "news_id": item["news_id"],
                "summary": None,
                "stock_list_view": None
            }
        else:
            row = {
                "news_id": item["news_id"],
                "summary": item.get("summary", ""),
                "stock_list_view": item.get("stock_list_view", [])
            }
        processed_data.append(row)
    
    return pd.DataFrame(processed_data)


In [ ]:
# # 함수 호출
# metadata_df = get_metadata_df(final_news_ids)
# metadata_df

,news_id,summary,stock_list_view
0,20250513_0089,호반건설이 지난 3월 한진칼 주주총회에 참석해 이사 보수 한도를 90억원에서 120...,"[{'stock_id': '180640', 'stock_name': '한진칼'}]"
1,20250513_0148,호반건설이 12일 한진칼 보유 지분을 17.44%에서 18.46%로 1.02%포인트...,"[{'stock_id': '003490', 'stock_name': '대한항공'},..."
2,20250513_0085,호반건설이 대한항공을 거느린 한진그룹 지주사인 한진칼 지분을 추가로 매입했다는 소식...,"[{'stock_id': '003490', 'stock_name': '대한항공'},..."
3,20250513_0041,호반건설이 한진칼 지분을 추가로 매입해 지분율(특수관계인 포함)이 17.44%에서 ...,"[{'stock_id': '180640', 'stock_name': '한진칼'}, ..."
4,20250430_0098,웅진이 국내 1위 상조 업체인 프리드라이프를 인수한다는 소식에 장 초반 급등하고 있...,"[{'stock_id': '016880', 'stock_name': '웅진'}]"
5,20250430_0083,웅진이 종속회사 WJ라이프가 상조회사 프리드라이프 지분 99.77%를 인수하는 주식...,"[{'stock_id': '016880', 'stock_name': '웅진'}, {..."
6,20250408_0068,더불어민주당은 노무현 전 대통령이 추진했지만 불발된 세종시 행정수도 이전 공약을 다...,"[{'stock_id': '013580', 'stock_name': '계룡건설'}]"
7,20250331_0086,한솔홀딩스의 한솔PNS 상장폐지를 위한 공개매수 소식이 알려지자 주가가 상승세를 보...,"[{'stock_id': '010420', 'stock_name': '한솔PNS'}..."
8,20250326_0027,이재명 더불어민주당 대표의 정치 테마주가 공직선거법 위반 사건 2심에서 무죄를 선고...,"[{'stock_id': '013360', 'stock_name': '일성건설'},..."
9,20250307_0027,법원이 고려아연 임시 주주총회 결의 중 집중투표제 도입만 효력을 유지하고 나머지 안...,"[{'stock_id': '000670', 'stock_name': '영풍'}, {..."


In [ ]:
# metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   news_id          17 non-null     object
 1   summary          17 non-null     object
 2   stock_list_view  17 non-null     object
dtypes: object(3)
memory usage: 540.0+ bytes


In [118]:
# news_id 기준으로 metadata_df와 merged_df 병합 (inner join)
final_df = pd.merge(metadata_df, merged_df, on='news_id', how="inner")

In [119]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   news_id          1 non-null      object
 1   summary          1 non-null      object
 2   stock_list_view  1 non-null      object
 3   timestamp        1 non-null      object
 4   timestampRaw     1 non-null      object
 5   thread           1 non-null      object
 6   level            1 non-null      object
 7   logger           1 non-null      object
 8   message          1 non-null      object
 9   memberId         1 non-null      object
 10  username         1 non-null      object
 11  userPnl          1 non-null      int64 
 12  asset            1 non-null      int64 
 13  investScore      1 non-null      int64 
 14  memberStocks     1 non-null      object
dtypes: int64(3), object(12)
memory usage: 252.0+ bytes


In [120]:
final_df

,news_id,summary,stock_list_view,timestamp,timestampRaw,thread,level,logger,message,memberId,username,userPnl,asset,investScore,memberStocks
0,20250513_0085,호반건설이 대한항공을 거느린 한진그룹 지주사인 한진칼 지분을 추가로 매입했다는 소식...,"[{'stock_id': '003490', 'stock_name': '대한항공'},...",2025-06-18T19:16:28,2025-06-18 19:16:28,http-nio-8080-exec-7,INFO,c.n.n.a.n.v2.impl.ml.GetDetailNewsV2,2025-06-18 19:16:28 [http-nio-8080-exec-7] [IN...,anonymous,anonymous,0,0,0,[]


광일님 이부분 지금은 DB 직접 조회하고있는데 api로 만들 때는 스키마에서 가져오게 해주시면 될 것 같아요

In [138]:
def get_final_prediction_df(final_df: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    """
    토픽 데이터 조회, 메인 토픽 추가, 주식 일치 계산, 최종 예측 데이터프레임 생성
    :param final_df: 최종 데이터프레임 (news_id, stock_list_view, memberStocks 컬럼 필수)
    :return: 
        - pred_df: 예측에 사용할 최종 데이터프레임
        - result_df: 모든 중간 처리 결과가 포함된 데이터프레임
    """
    # 1. 뉴스 ID 리스트 추출
    if 'news_id' not in final_df.columns:
        raise ValueError("final_df must contain 'news_id' column")
    news_ids = final_df['news_id'].tolist()
    
    # 2. 토픽 데이터 조회 (DB 연결 포함)
    topic_df = pd.DataFrame()
    try:
        # DB 연결
        conn = psycopg2.connect(
            host='3.37.207.16',
            port='5432',
            dbname='postgres',
            user='postgres',
            password='password'
        )
        cursor = conn.cursor()
        
        # 안전한 IN 절 쿼리
        query = """
        SELECT news_id, topic_1, topic_2, topic_3, topic_4, topic_5,
               topic_6, topic_7, topic_8, topic_9
        FROM news_v2_topic
        WHERE news_id IN %s
        """
        cursor.execute(query, (tuple(news_ids),))
        rows = cursor.fetchall()
        
        # DataFrame 생성
        columns = ['news_id', 'topic_1', 'topic_2', 'topic_3', 'topic_4',
                   'topic_5', 'topic_6', 'topic_7', 'topic_8', 'topic_9']
        topic_df = pd.DataFrame(rows, columns=columns)
        
    except Exception as e:
        print(f"Database error: {e}")
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()
    
    # 3. 메인 토픽 추가
    if not topic_df.empty:
        topic_columns = [f'topic_{i}' for i in range(1, 10)]
        has_topic_data = topic_df[topic_columns].notna().any(axis=1)
        
        if has_topic_data.any():
            main_topic_str = topic_df.loc[has_topic_data, topic_columns].idxmax(axis=1)
            main_topic_series = main_topic_str.str.extract(r'(\d+)', expand=False).astype(int)
            topic_df.loc[has_topic_data, 'main_topic'] = main_topic_series
    
    # 4. 토픽 데이터 병합
    if not topic_df.empty:
        result_df = final_df.merge(topic_df, on='news_id', how='left')
    else:
        result_df = final_df.copy()
        print("토픽 데이터 없음. 기본 컬럼 추가")
        for i in range(1, 10):
            result_df[f'topic_{i}'] = None
        result_df['main_topic'] = None
    
    # 5. 주식 일치 여부 계산
    def check_stock_overlap(row):
        """주식 리스트와 사용자 주식 비교 함수"""
        try:
            # 뉴스의 주식 ID 추출
            news_stocks = {stock['stock_id'] for stock in row['stock_list_view']}
            # 사용자 주식 코드 추출
            user_stocks = {stock['stockCode'] for stock in row['memberStocks']}
            # 교집합 확인
            return 1 if news_stocks & user_stocks else 0
        except (TypeError, KeyError):
            return 0
    
    # is_same_stock 컬럼 추가
    result_df['is_same_stock'] = result_df.apply(check_stock_overlap, axis=1)
    
    # 6. 최종 예측용 컬럼 선택
    columns_to_keep = [
        'userPnl', 'asset', 'investScore', 'topic_1', 'topic_2', 'topic_3', 'topic_4',
        'topic_5', 'topic_6', 'topic_7', 'topic_8', 'topic_9', 'main_topic',
        'is_same_stock', 'clicked'
    ]
    
    # 존재하는 컬럼만 선택
    available_columns = [col for col in columns_to_keep if col in result_df.columns]
    pred_df = result_df[available_columns].copy()
    
    # 누락된 컬럼 처리
    missing_columns = set(columns_to_keep) - set(available_columns)
    if missing_columns:
        print(f"누락된 컬럼: {', '.join(missing_columns)}")
        for col in missing_columns:
            if col == 'clicked':
                pred_df[col] = 0  # 클릭 정보 없으면 0 처리
            else:
                pred_df[col] = 0  # 기타 컬럼은 0으로 처리
    
    return pred_df, result_df  # 두 개의 DataFrame 반환


In [ ]:
# pred_df, result_df = get_final_prediction_df(final_df)
# pred_df

누락된 컬럼: clicked


,userPnl,asset,investScore,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,main_topic,is_same_stock,clicked
0,0,0,0,0.0078,0.0079,0.0078,0.0079,0.2139,0.0078,0.0078,0.0078,0.7313,9.0,0,0


In [ ]:
# result_df

,news_id,summary,stock_list_view,timestamp,timestampRaw,thread,level,logger,message,memberId,...,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,main_topic,is_same_stock
0,20250513_0085,호반건설이 대한항공을 거느린 한진그룹 지주사인 한진칼 지분을 추가로 매입했다는 소식...,"[{'stock_id': '003490', 'stock_name': '대한항공'},...",2025-06-18T19:16:28,2025-06-18 19:16:28,http-nio-8080-exec-7,INFO,c.n.n.a.n.v2.impl.ml.GetDetailNewsV2,2025-06-18 19:16:28 [http-nio-8080-exec-7] [IN...,anonymous,...,0.0079,0.0078,0.0079,0.2139,0.0078,0.0078,0.0078,0.7313,9.0,0


4. 나온 결과로 내 모델에 넣어서 리랭킹 4개

In [141]:
def get_final_recommendations(
    pred_df: pd.DataFrame, 
    result_df: pd.DataFrame, 
    top_k: int = None
) -> pd.DataFrame:
    """
    LightGBM 모델을 사용한 추천 시스템 전체 프로세스
    :param pred_df: 모델 예측용 데이터프레임 (피처 포함)
    :param result_df: 원본 데이터프레임 (추가 정보 포함)
    :param top_k: 반환할 상위 추천 개수
    :return: 최종 추천 결과 데이터프레임
    """
    # 1. 프로젝트 루트 경로 설정
    project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
    if project_root not in sys.path:
        sys.path.append(project_root)
    
    # 2. 모델 경로 정의
    possible_paths = [
        '../../modelapi/models/lgbm_model2.pkl',
        '../../../modelapi/models/lgbm_model2.pkl',
        'modelapi/models/lgbm_model2.pkl',
        os.path.join(project_root, 'modelapi/models/lgbm_model2.pkl')
    ]
    
    # 3. 모델 로드
    model = None
    for path in possible_paths:
        if os.path.exists(path):
            try:
                model = joblib.load(path)
                print(f"모델 로드 성공: {path}")
                break
            except Exception as e:
                print(f"모델 로드 실패 ({path}): {str(e)}")
    
    if model is None:
        print("사용 가능한 모델 파일을 찾을 수 없습니다.")
        return None
    
    # 4. 피처 데이터 준비
    features_df = pred_df.copy()
    
    # 'clicked' 컬럼이 있을 경우 제거 (추론 시 불필요)
    if 'clicked' in features_df.columns:
        features_df = features_df.drop(columns=['clicked'])
    
    print(f"피처 데이터 형태: {features_df.shape}")
    print(f"컬럼 목록: {list(features_df.columns)}")
    
    # 5. 예측 수행
    try:
        # 클릭 확률 예측 (클래스 1에 대한 확률)
        proba = model.predict_proba(features_df)[:, 1]
        
        # 예측 결과를 원본 데이터프레임에 병합
        recommendations = result_df.copy()
        recommendations['click_probability'] = proba
        
        # 6. 최종 데이터프레임 생성
        real_final_df = pd.concat(
            [result_df, recommendations[['click_probability']]],
            axis=1
        )
        
        # 7. 확률 기준 정렬
        real_final_df = real_final_df.sort_values(
            'click_probability', 
            ascending=False
        ).reset_index(drop=True)
        
        # 8. 상위 top_k개 선택
        if top_k is not None:
            real_final_df = real_final_df.head(top_k)
            print(f"상위 {top_k}개 추천 완료")
        else:
            print("전체 데이터에 대한 예측 완료")
        
        return real_final_df
    
    except Exception as e:
        print(f"예측 중 오류 발생: {str(e)}")
        return None


In [142]:
# 함수 실행 (상위 10개 추천)
final_recommendations = get_final_recommendations(
    pred_df=pred_df,
    result_df=result_df,
    top_k=10
)

# 결과 확인
if final_recommendations is not None:
    print("\n=== 최종 추천 결과 ===")
    print(final_recommendations[['news_id', 'click_probability']].head())
else:
    print("추천 실패")

모델 로드 성공: ../../modelapi/models/lgbm_model2.pkl
피처 데이터 형태: (1, 14)
컬럼 목록: ['userPnl', 'asset', 'investScore', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7', 'topic_8', 'topic_9', 'main_topic', 'is_same_stock']
상위 10개 추천 완료

=== 최종 추천 결과 ===
         news_id  click_probability
0  20250513_0085           0.546186


이제 최종 반환값은 click_probability 값 상위 4개 정도의 news_id를 뽑아내시면 됩니다!!